In [2]:
# import data, N(number of observation), N_f(number of features)
from _data import train_raw, N, N_f

In [7]:
import pandas as pd
import numpy as np

### Try PCA

## some ideas:
- create new cols store information by acct
- clustering the acct by avg_txn_amt, avg_month_amt, types of spending type, acc_code.....

#### even more ideas: 
- fit a dist model of txn time, txn number for every group --> give every txn a probability.
- try use one-class SVM (for novelty detection) see <a href="http://papers.nips.cc/paper/1723-support-vector-method-for-novelty-detection.pdf" > paper </a> for explain and exmple

### comput attribute by acct 

In [3]:
# compute unique value for each featue
def count_unique(df):
    unique_dict = {}
    for i in df.columns:
        uni_val = df[i].unique()
        unique_dict[i]=(len(uni_val), uni_val)
    return unique_dict
unique_dict = count_unique(train_raw)

In [8]:
bool_col=[]
for i in unique_dict:
    if unique_dict[i][0]<=3:
        bool_col.append(i)

bool_col.remove('fraud_ind')
bool_agg = {i:'sum' for i in bool_col}

col_nm_txn = ['tx_amt_avg', 'tx_cnt']
by_txn = train_raw.groupby('bacno').agg({'conam':['mean','count'], **bool_agg})
by_txn.columns=col_nm_txn+[i+'_count' for i in bool_col]
by_txn.replace(np.NAN, 0,inplace = True)

In [10]:
# count stscd_type for each acct
stscd_pvt = train_raw.pivot_table(index = ['bacno'], columns=['stscd'],values='conam',aggfunc='count')
stscd_pvt.replace(np.NAN, 0,inplace = True)
col_nm_stscd = ['stscd_'+str(i) for i in range(5) ]
stscd_pvt.columns = col_nm_stscd

by_txn = pd.concat([by_txn, stscd_pvt], axis=1)
by_txn.describe()

,tx_amt_avg,tx_cnt,ecfg_count,flbmk_count,flg_3dsmk_count,insfg_count,ovrlt_count,stscd_0,stscd_1,stscd_2,stscd_3,stscd_4,stscd_0,stscd_1,stscd_2,stscd_3,stscd_4
count,95214.000000,95214.000000,95214.000000,95214.000000,95214.000000,95214.000000,95214.000000,95214.000000,95214.000000,95214.000000,95214.000000,95214.000000,95214.000000,95214.000000,95214.000000,95214.000000,95214.000000
mean,732.571089,15.982807,4.067417,0.053280,0.675730,0.444388,0.212689,15.782816,0.004285,0.194362,0.001134,0.000210,15.782816,0.004285,0.194362,0.001134,0.000210
std,338.143649,21.017744,10.538025,0.336961,2.417233,2.196183,2.001265,20.658418,0.194617,1.786932,0.042734,0.025102,20.658418,0.194617,1.786932,0.042734,0.025102
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,537.374600,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000
50%,671.962735,9.000000,1.000000,0.000000,0.000000,0.000000,0.000000,9.000000,0.000000,0.000000,0.000000,0.000000,9.000000,0.000000,0.000000,0.000000,0.000000
75%,837.602885,20.000000,4.000000,0.000000,0.000000,0.000000,0.000000,20.000000,0.000000,0.000000,0.000000,0.000000,20.000000,0.000000,0.000000,0.000000,0.000000
max,6728.970000,1117.000000,406.000000,17.000000,147.000000,373.000000,169.000000,1117.000000,29.000000,124.000000,3.000000,6.000000,1117.000000,29.000000,124.000000,3.000000,6.000000


### clustering
    try to cluster accts into groups

#### 1. k-means
- result : 5 or 6 groups is good


In [ ]:
# K-means
from sklearn import cluster
inertia_record = []

X = by_txn
for n in range(1,15):
    k_means = cluster.KMeans(n_clusters=n)
    k_means.fit(X) 
    inertia_record.append(k_means.inertia_)

plt.plot([i for i in range(1,15)],inertia_record)


#### 2. novelty/outlier detection model

- one-class SMV
    
    > Hmm..... <br/>
    > not doing good"

In [12]:
# select wanted fearture and paramater
elements_col = np.setdiff1d(train_raw.columns.values, ['cano','bacno','fraud_ind','txkey'])
label_col = 'fraud_ind'
all_col = np.append(elements_col,label_col)
sample_fraction = 100

In [13]:
from sklearn.svm import OneClassSVM
sample_data = train_raw[all_col].sample((round(N/sample_fraction)))
train_x = sample_data[elements_col]
train_y = sample_data[label_col]
train_x.replace(np.NAN, 0,inplace = True)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [14]:
# compare model
kernel_options = ['rbf', 'sigmoid','poly']
kernel_options = [kernel_options[0]]
fun_compare = {}
for ko in kernel_options:
    clf = OneClassSVM(degree = 3, kernel=ko, gamma='auto', nu=20355/1501432).fit(train_x)
    pred = clf.predict(train_x)
    fun_compare[ko]=(pred, len(np.where(pred==1)[0]),len(np.where(pred==-1)[0]))
fun_compare

{'rbf': (array([-1, -1, -1, ...,  1,  1,  1]), 14121, 1097)}

In [79]:
def model_evl(pred, label):
    if len(pred)!=len(label):
        raise Exception('length not equal, plz chk, pred:%s, label: %s'%(len(pred), len(label)))
    evl = {}
    if sorted(np.unique(pred)) == [-1,1]:
        pred[pred==-1]=0
    a = pd.concat([pd.Series(pred, name='pred'), label.reset_index()['fraud_ind']],axis=1)
    #evl['ACC'] = sum(a['pred']==a['fraud_ind'])/len(pred) # accuracy
    evl['TP'] = sum((a['pred']==1) & (a['fraud_ind'] ==0)) # inlier & non-fraud
    evl['TN'] = sum((a['pred']==0) & (a['fraud_ind'] ==1)) # outlier & fraud
    evl['FA'] = sum((a['pred']==1) & (a['fraud_ind'] ==1)) # 
    evl['MS'] = sum((a['pred']==0) & (a['fraud_ind'] ==0)) # 
    evl['ACC'] = (evl['TP']+evl['TN'])/len(pred)
    return evl

In [80]:
for f in kernel_options:
    evl_f = model_evl(fun_compare[f][0], train_y)
    print(f,':', evl_f)
    

rbf : {'TP': 13933, 'TN': 5, 'FA': 188, 'MS': 1092, 'ACC': 0.9158890787225654}


----

- isolation forest

In [82]:

#try isolation forest
from sklearn.ensemble import IsolationForest
clf = IsolationForest(bootstrap=True,  max_samples=500, max_features=3, contamination=0.1, verbose=1, n_estimators=250)
clf.fit(train_x)

pred = clf.predict(train_x)
pred

model_evl(pred,train_y)


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s finished
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/stats/stats.py:1706: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


{'ACC': 0.9034695755026941, 'FA': 70, 'MS': 1399, 'TN': 123, 'TP': 13626}

In [73]:
type(a)

pandas.core.frame.DataFrame